### [将普通函数放入线程池执行await](https://github.com/lan2720/gpt-researcher/blob/5c55192c05fe834a0c52ffcf585446373bc2e3bb/actions/web_scrape.py#L57)
- async_browse函数内创建了8线程池
    - 将`scrape_text_with_selenium(url)`加入线程池
    - 将`add_header(driver)`加入线程池
    - 将`summary.summarize_text(url, text, question, driver)`加入线程池
    - 以上三个函数都是普通函数，不是async修饰的异步函数


In [1]:
import sys
sys.path.append("/Users/jarvixwang/Documents/Project/gpt-researcher/")

import asyncio
from concurrent.futures import ThreadPoolExecutor
from actions.web_scrape import scrape_text_with_selenium, add_header
import processing.text as summary
from config import Config
CFG = Config()

selenium_web_browser: firefox


In [5]:
loop = asyncio.get_event_loop()
# 定义了8线程的线程池
executor = ThreadPoolExecutor(max_workers=8)

url = "https://cn.nytimes.com/world/20231016/hamas-israel-mood-distrust/"
# url = "https://en.wikipedia.org/wiki/Large_language_model"
question = "please extract all related information about Israeli-Palestinian conflict. If not, find any possible relationship between the question and the given text."#"目前巴以冲突战况如何"

print(f"Scraping url {url} with question {question}")
# scrape_text_with_selenium(url)
# 它会将 func(*args) 函数添加到线程池(executor)中，并返回一个协程对象。
# 协程对象可以使用 await 关键字等待函数执行完成，并返回函数的结果（driver, text）
driver, text = await loop.run_in_executor(executor, scrape_text_with_selenium, url)
# 将add_header(driver)加入线程池: add_header(driver)是对正在分析的网页加一个遮罩层
await loop.run_in_executor(executor, add_header, driver)
# 将summary.summarize_text(url, text, question, driver)加入到线程池, 等待处理
summary_text = await loop.run_in_executor(executor, summary.summarize_text, url, text, question, driver, CFG.smart_llm_model)

Scraping url https://cn.nytimes.com/world/20231016/hamas-israel-mood-distrust/ with question please extract all related information about Israeli-Palestinian conflict. If not, find any possible relationship between the question and the given text.
当前选择的浏览器是: firefox
CFG.selenium_web_browser:  firefox
****************创建了firefox的driver
scraping url https://cn.nytimes.com/world/20231016/hamas-israel-mood-distrust/...
Summarizing url: https://cn.nytimes.com/world/20231016/hamas-israel-mood-distrust/ with total chunks: 1
Final summary length:  752
combined_summary: The text does not provide specific details about the Israeli-Palestinian conflict. However, it mentions that Isabel Kershner, a New York Times correspondent based in Jerusalem, has been reporting on Israeli and Palestinian politics since 1990. Her latest book is titled "The Land of Hope and Fear: Israel’s Battle for its Inner Soul". The text also refers to several articles related to Israel, including one about why Israeli securi

In [7]:
summary_text

'The text does not provide specific information about the current situation of the Israeli-Palestinian conflict.'